In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
import pandas as pd
import numpy as np

In [27]:
data = pd.read_csv('/content/gdrive/MyDrive/Kaggle/Copy of princess_corpus.csv')

In [28]:
import nltk
nltk.download('punkt')

from nltk import word_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('russian')

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('russian')

def custom_tokenizer(text):
  return [stemmer.stem(w) for w in word_tokenize(text.lower(), language='russian') if w.isalpha() and not w in sw]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
sents = sum([nltk.sent_tokenize(x) for x in data.Text], [])
data["n_sent"] = data["Text"].apply(lambda x: len(nltk.sent_tokenize(x)))
data["target"] = data.apply(lambda x: [x.Speaker_Status]*x.n_sent, axis=1)
target = sum(data["target"].tolist(), [])

In [30]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

hash_vec3 = HashingVectorizer(tokenizer=custom_tokenizer, n_features=10000)
train_hash = hash_vec3.fit_transform(sents)

clf2 = SGDClassifier(random_state=0)
clf2.fit(train_hash, target)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=0, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [31]:
from sklearn.metrics import accuracy_score

print(np.round(accuracy_score(clf2.predict(train_hash), target), decimals=3))

0.675


In [32]:
from joblib import dump, load
dump(hash_vec3, 'vectorizer.joblib') 

['vectorizer.joblib']

In [33]:
vectorizer = load('vectorizer.joblib') 

In [34]:
dump(clf2, 'clf.joblib')
clf = load('clf.joblib')

In [37]:
def answer(text):
  a = clf.predict(vectorizer.transform([sents[0]])).tolist()[0]
  if a == "PRINCESS":
    return "You are a princess!"
  elif a == "PRINCE":
    return "You are a prince."
  else:
    return "You are a side character:("
  

In [51]:
df = pd.DataFrame(clf2.predict(train_hash))
df[df[0] == "PRINCE"]

,0
695,PRINCE
1443,PRINCE
2070,PRINCE
2185,PRINCE
2247,PRINCE
...,...
12088,PRINCE
13048,PRINCE
13072,PRINCE
13169,PRINCE


In [61]:
n = int(input())
print(sents[n], target[n])

12001
i tore a great rift in our kingdom. PRINCESS
